# 📝 Exercise M3.01

The goal is to write an exhaustive search to find the best parameters
combination maximizing the model statistical performance.

Here we use a small subset of the Adult Census dataset to make the code
faster to execute. Once your code works on the small subset, try to
change `train_size` to a larger value (e.g. 0.8 for 80% instead of
20%).

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("../datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.8, random_state=42)

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer(
    [('cat-preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)

# This line is currently required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))
])


Use the previously defined model (called `model`) and using two nested `for`
loops, make a search of the best combinations of the `learning_rate` and
`max_leaf_nodes` parameters. In this regard, you will need to train and test
the model by setting the parameters. The evaluation of the model should be
performed using `cross_val_score`. We will use the following parameters
search:
- `learning_rate` for the values 0.01, 0.1, 1 and 10. This parameter controls
  the ability of a new tree to correct the error of the previous sequence of
  trees
- `max_leaf_nodes` for the values 3, 10, 30. This parameter controls the
  depth of each tree.

In [3]:
model.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                     transformers=[('cat-preprocessor',
                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                   unknown_value=-1),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000002A8E4E857C0>)])),
  ('classifier', HistGradientBoostingClassifier(random_state=42))],
 'verbose': False,
 'preprocessor': ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                   transformers=[('cat-preprocessor',
                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                 unknown_value=-1),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002A8E4E857C0>)]),
 'classifier': HistGradientBoostingClassifier(r

In [4]:
from sklearn.model_selection import cross_val_score

learning_rates = [0.01, 0.1, 1, 10]
max_leaf_nodes = [3, 10, 30]
for mln in max_leaf_nodes:
    for lr in learning_rates:
        model.set_params(
            classifier__learning_rate=lr,
            classifier__max_leaf_nodes=mln
        )
        score = cross_val_score(model, data_train, target_train)
        print(f'Learning rate: {lr} - Max leaf node: {mln} - Accuracy: {score.mean()}')

Learning rate: 0.01 - Max leaf node: 3 - Accuracy: 0.7974559991327475
Learning rate: 0.1 - Max leaf node: 3 - Accuracy: 0.8545542598623367
Learning rate: 1 - Max leaf node: 3 - Accuracy: 0.8617458206565607
Learning rate: 10 - Max leaf node: 3 - Accuracy: 0.28234362557091536
Learning rate: 0.01 - Max leaf node: 10 - Accuracy: 0.8187750286941707
Learning rate: 0.1 - Max leaf node: 10 - Accuracy: 0.8691934600380143
Learning rate: 1 - Max leaf node: 10 - Accuracy: 0.8624109784740908
Learning rate: 10 - Max leaf node: 10 - Accuracy: 0.7612419298923909
Learning rate: 0.01 - Max leaf node: 30 - Accuracy: 0.8483095207332475
Learning rate: 0.1 - Max leaf node: 30 - Accuracy: 0.8714456736526677
Learning rate: 1 - Max leaf node: 30 - Accuracy: 0.8592121036753266
Learning rate: 10 - Max leaf node: 30 - Accuracy: 0.3564164030602094
